In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
import seaborn as sns
sns.set(color_codes=True)
import geopandas as gp
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [3]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [4]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [5]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [6]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [7]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [8]:
#Visualization on how the function works
us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']

herepath['DATE'] = herepath['DATE_TIME'].str.split(' ').str[0]
r = herepath[herepath['DATE'].isin(us_holidays)]
v = herepath[~herepath.isin(r)]
w = v.dropna()
r.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,0,998969850F,2014-01-01 13:55,167,186,61.0,0.0,10,Wednesday,2014-01-01
1099,1099,998969850F,2014-05-26 11:25,137,186,64.5,3.5,30,Monday,2014-05-26
1100,1100,998969850F,2014-05-26 14:15,171,186,76.0,0.0,10,Monday,2014-05-26
1101,1101,998969850F,2014-05-26 15:05,181,186,65.0,0.0,10,Monday,2014-05-26
1102,1102,998969850F,2014-05-26 17:25,209,186,31.0,0.0,10,Monday,2014-05-26


In [9]:
v.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,998969850F,2014-01-02 11:05,133.0,186.0,60.0,0.0,10.0,Thursday,2014-01-02
2,2.0,998969850F,2014-01-02 13:20,160.0,186.0,61.0,0.0,10.0,Thursday,2014-01-02
3,3.0,998969850F,2014-01-02 14:30,174.0,186.0,52.0,0.0,10.0,Thursday,2014-01-02
4,4.0,998969850F,2014-01-03 09:05,109.0,186.0,57.0,0.0,10.0,Friday,2014-01-03


In [10]:
#Proves the data doesn't have any NaN's other than what I specify to drop holidays
np.isnan(herepath.MEAN).unique()

array([False], dtype=object)

In [34]:
numberdays = data['DATE'].unique()
len(numberdays)

261

In [37]:
data = timeframe2(herepath,0,288,True,Weekday)
epoct = data['LINK_DIR'].value_counts()
epoct
epoct/(288*len(numberdays))

107030477T    0.221610
111990719T    0.213136
111990715T    0.212298
111999151F    0.208440
111990720F    0.200378
111990717F    0.197571
107030487T    0.195855
107034547T    0.193660
107030561T    0.192436
107373569T    0.192396
23320565T     0.188524
111996193T    0.182325
775719632T    0.182099
107030485T    0.181753
107373570F    0.181380
775719631T    0.180423
107030486T    0.179199
107030488F    0.178054
107030468T    0.177482
27593407F     0.177137
111986960F    0.173066
111999150F    0.170644
107030568T    0.170378
111989989T    0.168636
107030483F    0.168409
111990748T    0.167372
111990747T    0.165682
107030484F    0.165509
107030465T    0.161478
107030466T    0.161438
                ...   
111981912T    0.070855
997244941T    0.069444
23320410T     0.069325
107034534T    0.068952
997244940T    0.068034
27616976T     0.067688
27616977T     0.067436
107030432T    0.066824
111986235T    0.066544
111938004T    0.064083
23320387F     0.063471
111987140F    0.055901
107410437F 

In [8]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstats(x):
    print('Mean is ', np.mean(x['MEAN']), 'KMH')
    var = ((x['MEAN'].var(ddof=1))+(((x['STDDEV'])**2).sum()/len(x)))
    print('Standard Deviation is ', np.sqrt(var),'KMH')
    print('85th percentile speed is ', x['MEAN'].quantile(0.85,interpolation = 'lower' ), 'KMH')
    print('5th percentile speed is ', x['MEAN'].quantile(0.05,interpolation = 'lower'), 'KMH')
    
    ax = sns.distplot(x['MEAN'], kde=False, bins=70)
    ax.figure.set_size_inches(8,6)
    ax.set_xlim(0,100)
    return ax

In [8]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsGB(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWKD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWKND(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWDTime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsGB(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsGB(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))*(0.6213)
    
    y['85spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    
    numberdays = x['DATE'].unique()
    print(len(numberdays))
    epoct = x['LINK_DIR'].value_counts()
    y['TotalInterval'] = epoct
    y['PcntInterval'] = ((epoct/(288*len(numberdays))*100))
    
    y = np.round(y,3)
    return y

In [98]:
#For all days and all time periods
bylink = HEREstatsGB(herepath)
bylink.head()

365


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
107030432T,38.553,10.444,48.461,42.248,19.260,49.704,1.289,2.581,1.257,2.516,1.096,2.194,1.002,5746,5.466
107030439T,35.148,9.929,44.734,38.521,17.396,44.734,1.273,2.571,1.273,2.571,1.096,2.214,1.020,7857,7.474
107030440T,34.863,10.303,45.355,38.521,17.086,44.734,1.283,2.618,1.301,2.655,1.105,2.255,1.040,7325,6.968
107030444F,36.180,11.209,47.219,40.695,16.154,44.734,1.236,2.769,1.305,2.923,1.125,2.519,1.240,9311,8.857
107030445F,36.602,11.250,47.840,41.627,16.775,44.734,1.222,2.667,1.307,2.852,1.137,2.481,1.182,9256,8.805


In [103]:
#for weekdays
Wkd = timeframe('998969850F',herepath,0,288,True,Weekday)
HEREstatsGB(Wkd).head()

254


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,33.964,9.22,42.87,36.657,18.018,44.734,1.317,2.483,1.262,2.379,1.079,2.034,0.885,2799,3.826


In [74]:
#for weekends
Wend = timeframe('998969850F',herepath,0,288,True,Weekend)
HEREstatsGB(Wend).head()

99


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,36.202398,10.750527,46.5975,39.1419,17.3964,44.7336,1.235653,2.571429,1.287138,2.678571,1.081196,2.25,1.081028,330,1.157407


In [75]:
#Weekday Between 6AM and 8PM
Wdtime = timeframe('998969850F',herepath,72,240,True,Weekday)
HEREstatsGB(Wdtime).head()

254


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,33.292578,8.937691,41.6271,36.0354,18.0177,44.7336,1.343651,2.482759,1.250342,2.310345,1.082385,2.0,0.847771,2587,3.536472


In [76]:
#Stats between 6AM and 9AM
WdtimeAM = timeframe('998969850F',herepath,72,108,True,Weekday)
HEREstatsGB(WdtimeAM).head()

210


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,33.837551,8.697998,42.2484,36.6567,18.0177,44.7336,1.322011,2.482759,1.248566,2.344828,1.083314,2.034483,0.878017,440,0.727513


In [77]:
#Stats between 9AM and 3PM
WdtimeMD = timeframe('998969850F',herepath,108,180,True,Weekday)
HEREstatsGB(WdtimeMD).head()

250


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,34.01438,8.660703,42.2484,36.6567,18.639,44.7336,1.315138,2.4,1.242075,2.266667,1.077682,1.966667,0.824904,1367,1.898611


In [78]:
#Stats between 3PM and 6PM
WdtimePM = timeframe('998969850F',herepath,180,216,True,Weekday)
HEREstatsGB(WdtimePM).head()

235


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
998969850F,31.642208,9.324248,40.3845,34.1715,16.7751,44.7336,1.413732,2.666667,1.276286,2.407407,1.079934,2.037037,0.88626,651,0.961879
